# Week 8 – Autopilot Algorithm Development 🤖✈️

Focus: Guidance, Navigation & Control (GNC) fundamentals and a basic autopilot simulation.

---

### ✅ Learning Goals
- Understand GNC blocks: guidance (path planning), navigation (state estimation), control (actuators)
- Implement waypoint-following (simple proportional guidance)
- Simulate autopilot phases: takeoff → climb → cruise → descent → landing


In [ ]:
import numpy as np
import matplotlib.pyplot as plt


## 1) Simple Waypoint Following (2D)

We implement a proportional guidance controller to steer a vehicle to sequential waypoints.

In [ ]:
# Waypoint follower (2D) - simple proportional controller
def waypoint_follower(waypoints, dt=0.1, vmax=50.0, kp=0.5):
    pos = np.array(waypoints[0], dtype=float)  # start at first waypoint
    traj = [pos.copy()]
    vel = np.zeros(2)
    idx = 1
    while idx < len(waypoints):
        target = np.array(waypoints[idx], dtype=float)
        error = target - pos
        dist = np.linalg.norm(error)
        if dist < 1.0:  # reached waypoint
            idx += 1
            continue
        # Proportional steering towards waypoint
        direction = error / (dist + 1e-9)
        accel = kp * error  # simple proportional law
        vel = vel + accel*dt
        # limit speed
        speed = np.linalg.norm(vel)
        if speed > vmax:
            vel = vel / speed * vmax
        pos = pos + vel*dt
        traj.append(pos.copy())
        # safety break (avoid infinite loops)
        if len(traj) > 20000:
            break
    return np.array(traj)

# Example waypoints (x,y) meters
wps = [(0,0),(500,200),(1200,500),(2000,400),(3000,0)]
traj = waypoint_follower(wps, dt=0.5, vmax=100.0, kp=0.002)

plt.figure(figsize=(8,5))
wps_arr = np.array(wps)
plt.plot(traj[:,0], traj[:,1], label='Trajectory')
plt.scatter(wps_arr[:,0], wps_arr[:,1], color='red', marker='x', label='Waypoints')
plt.xlabel('X (m)')
plt.ylabel('Y (m)')
plt.title('Simple Waypoint Following Trajectory')
plt.legend()
plt.grid(True)
plt.axis('equal')
plt.show()

## 2) Autopilot Phases Simulation (simplified vertical profile)

We'll simulate altitude profile through phases: takeoff, climb, cruise, descent using simple setpoints and PID-like control (re-using the PID class from Week 7).

In [ ]:
# Simple altitude phase controller (uses proportional control)
def altitude_profile_sim(phases, dt=0.5):
    # phases: list of (phase_name, duration_s, target_altitude_m)
    t = []
    alt = []
    time = 0.0
    altitude = 0.0
    Kp = 0.5
    for name, duration, target in phases:
        steps = int(duration/dt)
        for _ in range(steps):
            error = target - altitude
            u = Kp * error  # control effort ~ climb rate
            # simple dynamics: altitude_dot = u (m/s) but limit climb rate
            climb_rate = np.clip(u, -20, 20)
            altitude = altitude + climb_rate*dt
            time += dt
            t.append(time)
            alt.append(altitude)
    return np.array(t), np.array(alt)

phases = [
    ("Takeoff", 30, 1000),
    ("Climb", 300, 10000),
    ("Cruise", 600, 10000),
    ("Descent", 400, 2000),
    ("Landing", 60, 0)
]

t, alt = altitude_profile_sim(phases, dt=1.0)
plt.figure(figsize=(10,4))
plt.plot(t, alt)
plt.xlabel('Time (s)')
plt.ylabel('Altitude (m)')
plt.title('Simplified Autopilot Vertical Profile')
plt.grid(True)
plt.show()

## 3) Navigation note: state estimation

For real autopilots, navigation uses sensor fusion (e.g., Kalman Filter) combining IMU and GPS to estimate position, velocity, attitude. Implementing a full KF is in Week 6/10 advanced modules.